# Introduction à la planification classique

## A. Présentation des technologies utilisées

### 1. Fast Downward

[Fast Downward](https://github.com/aibasel/downward) est un système de planification automatisée open-source développé en C++. Il implémente différents algorithmes de planification basés sur la recherche heuristique, tels que LAMA, Dijkstra, A*, etc. Fast Downward est utilisé pour résoudre des problèmes de planification dans divers domaines, tels que la planification de tâches, la planification logistique, la planification de ressources, etc.

### 2. Google OR-Tools

Google OR-Tools est une suite open-source de logiciels d'optimisation qui comprend des solvers pour les problèmes de programmation par contraintes (CSP), la satisfaction de contraintes booléennes (SAT), l'optimisation linéaire (LP), l'optimisation en nombres entiers (MIP), l'optimisation de réseau, et la planification de routes.
Dans le contexte de ce TP, nous nous intéresserons principalement aux fonctionnalités de planification de OR-Tools, qui fournissent une suite d'algorithmes et de méthodes pour résoudre des problèmes de planification tels que le problème du voyageur de commerce, le problème de sac à dos, et la planification de tâches.
OR-Tools peut être utilisé en C# via le package Nuget officiel. Il fournit une API C# qui permet d'utiliser les solvers et les méthodes d'optimisation de OR-Tools dans un projet .NET.


## B. Problèmes et domaines de planification

### 1. Qu’est-ce que la planification ?

La planification est une composante essentielle de l'intelligence artificielle (IA). Dans ce cadre, on peut définir la planification comme le processus permettant de déterminer une séquence d'actions qui mènera un agent d'un état initial à un état d'objectif. Ce concept est crucial dans diverses applications de l'IA, notamment dans la robotique, où il est souvent nécessaire d'établir un plan d'action pour un robot qui doit accomplir une série de tâches dans un ordre précis.

La planification en IA se base sur une représentation formalisée d'un problème de planification classique, qui est généralement exprimé comme un triplet : <I, G, D>. Ici, I représente l'état initial de l'agent, G est l'état d'objectif que l'agent veut atteindre, et D décrit la théorie du domaine, c'est-à-dire l'ensemble des actions que l'agent peut entreprendre pour modifier son environnement.

### 2. Représentation élémentaire : le modèle STRIPS

L'un des modèles de planification les plus fondamentaux en IA est STRIPS (Stanford Research Institute Problem Solver). Dans STRIPS, les actions sont formalisées avec une liste de préconditions et une liste d'effets. Les préconditions doivent être satisfaites pour que l'action puisse être effectuée, et les effets sont les conséquences de l'action sur l'état du monde.
Une action est définie par :
- Son nom
- Ses paramètres
- Ses préconditions
- Ses résultats

#### Exemple de STRIPS

Considérons un exemple simple dans le monde des blocs. Supposons que nous ayons un bras robotique qui peut effectuer un certain nombre d'actions spécifiques, comme prendre un bloc, le déposer, le placer sur un autre bloc, ou le retirer d'un autre bloc. Dans STRIPS, chaque action serait représentée avec une liste de préconditions et une liste d'effets. Par exemple, pour l'action de prendre un bloc, les préconditions pourraient inclure le fait que le bras soit libre et que le bloc soit accessible. Les effets de l'action seraient que le bras n'est plus libre (il tient le bloc) et que le bloc n'est plus accessible (il est dans la main du robot).

##### Quatre actions définies pour le monde des blocs

```strips
pickup(X)
P: gripping() ∧ clear(X) ∧ ontable(X)
A: gripping(X)
D: ontable(X) ∧ gripping()
putdown(X)
P: gripping(X)
A: ontable(X) ∧ gripping() ∧ clear(X)
D: gripping(X)
stack(X,Y)
P: gripping(X) ∧ clear(Y)
A: on(X,Y) ∧ gripping() ∧ clear(X)
D:gripping(X) ∧ clear(Y)
unstack(X,Y)
P: gripping() ∧ clear(X) ∧ on(X,Y)
A: gripping(X) ∧ clear(Y)
D: on(X,Y) ∧ gripping()
```

### 3. Les limites de STRIPS


STRIPS repose sur plusieurs hypothèses importantes qui simplifient le problème de planification, mais qui peuvent également limiter son applicabilité. Ces hypothèses comprennent l'idée que le monde est statique (à part les actions de l'agent), que toutes les actions sont déterministes et que l'agent a une connaissance parfaite du monde. Cependant, malgré sa utilité, STRIPS présente certaines limites.

#### Le problème du cadre

Le problème du cadre est l'un des problèmes qui peut se poser lors de l'utilisation de STRIPS. Il consiste à énumérer tout ce qui ne change pas dans le problème de planification. Cependant, de nos jours, ce problème est moins significatif car ce qui n'est pas explicitement listé comme changeant (via les listes d'ajout et de suppression d'effets) est supposé ne pas changer.

#### Le problème de qualification

Le problème de qualification est un autre défi dans la représentation des actions dans STRIPS. Il concerne la possibilité d'énumérer chaque précondition nécessaire pour qu'une action puisse être exécutée. Par exemple, dans le cas de l'action "prendre un bloc", il peut être nécessaire de spécifier que le bloc ne doit pas être collé à la table ou cloué à la table, etc. Ce problème reste une difficulté, mais une solution partielle consiste à hiérarchiser les préconditions en déterminant celles qui sont les plus importantes à atteindre.

#### Extensions de STRIPS : PDDL

Pour surmonter ces limites, le langage de définition du domaine de planification (PDDL) a été introduit. PDDL est une extension de STRIPS qui permet une plus grande flexibilité dans la représentation des problèmes de planification. Par exemple, PDDL permet de représenter les actions concurrentes, les actions non déterministes et les négations des effets des actions. De plus, PDDL offre des fonctionnalités supplémentaires, comme la possibilité de représenter les préférences et les contraintes temporelles.

### 4. PDDL et le problème du Gripper


Dans PDDL, chaque action ou opérateur est défini par son nom, ses paramètres, ses préconditions et ses effets. C'est une représentation spéciale conçue pour la planification qui simplifie considérablement le processus en faisant de nombreuses suppositions.


#### Domaine (domain.pddl)

```pddl
(define (domain gripper-strips)
  (:predicates (room ?r)
               (ball ?b)
               (gripper ?g)
               (at-robby ?r)
               (at ?b ?r)
               (free ?g)
               (carry ?o ?g))
  (:action move
           :parameters (?from ?to)
           :precondition (and (room ?from) (room ?to) (at-robby ?from))
           :effect (and (at-robby ?to) (not (at-robby ?from))))
  (:action pick
           :parameters (?obj ?room ?gripper)
           :precondition (and (ball ?obj) (room ?room) (gripper ?gripper) (at ?obj ?room) (at-robby ?room) (free ?gripper))
           :effect (and (carry ?obj ?gripper) (not (at ?obj ?room)) (not (free ?gripper))))
  (:action drop
           :parameters (?obj ?room ?gripper)
          

 :precondition (and (ball ?obj) (room ?room) (gripper ?gripper) (at-robby ?room) (carry ?obj ?gripper))
           :effect (and (at ?obj ?room) (free ?gripper) (not (carry ?obj ?gripper)))))
```

#### Problème (problem.pddl)

```pddl
(define (problem strips-gripper2)
  (:domain gripper-strips)
  (:objects rooma roomb ball1 ball2 left right)
  (:init (room rooma) (room roomb) (ball ball1) (ball ball2) (gripper left) (gripper right) (at-robby rooma) (free left) (free right) (at ball1 rooma) (at ball2 rooma))
  (:goal (at ball1 roomb)))
```


## C. Exécution de commandes avec Fast Downward

### Préparation de l'environnement
Nous allons créer une méthode en C# pour exécuter des commandes externes comme `fast-downward.py`. Cette méthode utilisera `System.Diagnostics.Process`.

In [11]:
using System;
using System.Diagnostics;
using System.IO;

public class FastDownwardRunner
{
    private static string FastDownwardPath = @"E:\Dev\AI\Libs\downward\fast-downward.py";  // Mettez ici le chemin absolu vers fast-downward.py
    private string BasePath { get; }

    public FastDownwardRunner(string basePath)
    {
        BasePath = basePath;
    }

    public void RunFastDownward(string domainFileRelative, string problemFileRelative, string additionalArgs = "")
    {
        var domainFile = Path.Combine(BasePath, domainFileRelative);
        var problemFile = Path.Combine(BasePath, problemFileRelative);

        // Afficher le début du fichier de domaine
        DisplayFileStart(domainFile, "Domaine");

        // Afficher le début du fichier de problème
        DisplayFileStart(problemFile, "Problème");

        var processStartInfo = new ProcessStartInfo
        {
            FileName = "python",
            Arguments = $"\"{FastDownwardPath}\" \"{domainFile}\" \"{problemFile}\" {additionalArgs}",
            RedirectStandardOutput = true,
            UseShellExecute = false,
            CreateNoWindow = true
        };

        using (var process = Process.Start(processStartInfo))
        {
            if (process != null)
            {
                using (var reader = process.StandardOutput)
                {
                    string result = reader.ReadToEnd();
                    Console.WriteLine(result);
                }
            }
        }

        // Afficher le contenu du fichier sas_plan
        var sasPlanFile = Path.Combine(Environment.CurrentDirectory, "sas_plan");
        if (File.Exists(sasPlanFile))
        {
            Console.WriteLine("Contenu de sas_plan:");
            Console.WriteLine(File.ReadAllText(sasPlanFile));
        }
        else
        {
            Console.WriteLine("Le fichier sas_plan n'a pas été trouvé.");
        }
    }

    private void DisplayFileStart(string filePath, string fileType)
    {
        if (File.Exists(filePath))
        {
            Console.WriteLine($"Début du fichier {fileType} ({filePath}):");
            using (var reader = new StreamReader(filePath))
            {
                for (int i = 0; i < 50; i++)  // Afficher les 50 premières lignes
                {
                    if (reader.EndOfStream) break;
                    Console.WriteLine(reader.ReadLine());
                }
            }
        }
        else
        {
            Console.WriteLine($"Le fichier {fileType} ({filePath}) n'a pas été trouvé.");
        }
    }

}

// Exemple d'initialisation de la classe et d'appel de la méthode avec des chemins relatifs
var runner = new FastDownwardRunner(@"E:\Dev\AI\Libs\downward");




## III. Syntaxe de ligne de commande de Fast Downward

Fast Downward propose une interface en ligne de commande flexible pour paramétrer ses différents composants. Les paramètres peuvent être spécifiés par mot-clé ou par position, et certains ont des valeurs par défaut. Voici quelques exemples de commandes et options disponibles.

Fast Downward propose une fonctionnalité permettant de spécifier différentes options en fonction des propriétés du fichier d'entrée, comme la sélection des heuristiques à utiliser en fonction du coût unitaire ou non unitaire des actions. Cette option est conditionnée par des sélecteurs tels que --if-unit-cost.

Fast Downward offre également la possibilité de définir des variables pour des objets qui doivent être utilisés à plusieurs reprises, par exemple une heuristique ou une LandmarkFactory. Ces variables peuvent être définies avec let(name, definition, expression).

Les options de commande comprennent des options pour sélectionner les composants du planificateur à exécuter (translation, recherche), des limites de temps et de mémoire, des options pour le fichier de plan et de nombreuses autres options.


### Options de traduction

Fast Downward permet une grande flexibilité dans la façon dont le problème de planification est traduit en une tâche de planification abstraite qui peut être résolue par l'algorithme de recherche. Les options de traduction permettent de spécifier des options telles que l'encodage complet et d'autres options spécifiques à la tâche de planification.

In [12]:
runner.RunFastDownward("misc/tests/benchmarks/gripper/domain.pddl", 
    "misc/tests/benchmarks/gripper/prob01.pddl", "--translate-options --full-encoding");

### Options de recherche

Les options de recherche spécifient l'algorithme de recherche à utiliser pour résoudre la tâche de planification. Fast Downward supporte une grande variété d'algorithmes de recherche, y compris A*, chaînage avant, chaînage arrière et planification d'ordre partiel. 

Les options de recherche permettent également de spécifier des options spécifiques à l'algorithme de recherche, telles que les heuristiques à utiliser.


In [13]:
// Exemple d'exécution avec les options de recherche
runner.RunFastDownward("misc/tests/benchmarks/gripper/domain.pddl", 
"misc/tests/benchmarks/gripper/prob01.pddl", 
"--search-options --heuristic \"h=ff()\" --search \"eager_greedy([h])\"");


### Utilisation d'un alias

Fast Downward permet l'utilisation d'alias pour spécifier des configurations de planification prédéfinies. Ces alias peuvent être spécifiés à l'aide de l'option --alias sur la ligne de commande.


In [14]:
// Exemple d'exécution avec des limites de temps et de mémoire
runner.RunFastDownward("misc/tests/benchmarks/gripper/domain.pddl", "", "--translate-time-limit 300 --translate-memory-limit 4000 --search-time-limit 500 --search-memory-limit 8000");


### Limites de temps et de mémoire

Fast Downward permet de spécifier des limites de temps et de mémoire pour les différentes composantes du planificateur. Ces limites peuvent être spécifiées à l'aide des options de ligne de commande --translate-time-limit, --translate-memory-limit, --search-time-limit, et --search-memory-limit.


In [15]:
// Exemple d'exécution avec des limites de temps et de mémoire
runner.RunFastDownward("misc/tests/benchmarks/gripper/domain.pddl", "", "--translate-time-limit 300 --translate-memory-limit 4000 --search-time-limit 500 --search-memory-limit 8000");


### Validation de plan

Fast Downward peut également valider les plans générés en utilisant l'outil de validation VAL.

### Utilisation d'un portfolio

Fast Downward supporte l'utilisation de portfolios de configurations de planification. Un portfolio est un ensemble de configurations de planification qui sont exécutées en séquence ou en parallèle pour résoudre une tâche de planification. L'utilisation d'un portfolio peut être spécifiée à l'aide de l'option --portfolio sur la ligne de commande.


In [16]:
// Exemple d'exécution avec un portfolio
runner.RunFastDownward("", "", "--portfolio driver/portfolios/seq_opt_fdss_1.py --search-time-limit 30m output.sas");



### Heuristiques avancées de Fast Downward

L'efficacité de Fast Downward réside dans son usage d'heuristiques de pointe pour guider la recherche de solutions aux problèmes de planification. Quelques-unes de ces heuristiques avancées sont décrites ci-dessous.

Chacune de ces heuristiques possède des avantages et des limites en termes d'efficacité, d'admissibilité, de cohérence, et d'identification d'opérateurs privilégiés. Le choix de l'heuristique à utiliser dépend donc du problème de planification spécifique à résoudre.


#### Additive Heuristic

L'heuristique additive (add) est une méthode permettant de combiner les estimations de plusieurs heuristiques pour une meilleure estimation. Elle n'est ni admissible, ni cohérente, mais identifie les opérateurs privilégiés. 

Les options de cette heuristique incluent le niveau de verbosité, la transformation de la tâche, et le cache des estimations.

In [17]:
// Exemple d'exécution avec l'heuristique additive
runner.RunFastDownward("misc/tests/benchmarks/gripper/domain.pddl", "misc/tests/benchmarks/gripper/prob01.pddl", "--search \"astar(add ())\"");


#### Blind Heuristic

L'heuristique aveugle (blind) retourne le coût de l'action la moins chère pour les états non-cibles et 0 pour les états cibles. Elle supporte les coûts d'action, les effets conditionnels, et les axiomes. C'est une heuristique admissible, cohérente et sûre, mais elle ne permet pas d'identifier des opérateurs privilégiés.

In [18]:
// Exemple d'exécution avec l'heuristique aveugle
runner.RunFastDownward("misc/tests/benchmarks/gripper/domain.pddl", "misc/tests/benchmarks/gripper/prob01.pddl", "--search \"astar(blind())\"");



#### Context-enhanced Additive Heuristic

L'heuristique context-enhanced additive (cea) est une heuristique non admissible, non cohérente et non sûre, mais qui identifie les opérateurs privilégiés. Comme pour les heuristiques précédentes, elle offre des options pour le niveau de verbosité, la transformation de la tâche, et le cache des estimations.

In [19]:
// Exemple d'exécution avec l'heuristique context-enhanced additive
runner.RunFastDownward("misc/tests/benchmarks/gripper/domain.pddl", "misc/tests/benchmarks/gripper/prob01.pddl", "--search \"astar(cea())\"");


#### Merge-and-shrink Heuristic

L'heuristique merge-and-shrink (merge_and_shrink) est basée sur l'algorithme décrit dans plusieurs articles scientifiques. Cette heuristique est configurable avec plusieurs stratégies de fusion et de réduction. Elle supporte les coûts d'action et les effets conditionnels, mais pas les axiomes. Elle est admissible, cohérente, sûre, mais ne permet pas d'identifier des opérateurs privilégiés.

In [20]:
// Exemple d'exécution avec l'heuristique merge-and-shrink
runner.RunFastDownward("misc/tests/benchmarks/gripper/domain.pddl", "misc/tests/benchmarks/gripper/prob01.pddl", "--search \"astar(merge_and_shrink ())\"");


#### Operator-counting Heuristic

L'heuristique operator-counting (operatorcounting) est une heuristique qui calcule un programme linéaire à chaque état. Elle est admissible, cohérente, sûre, mais ne permet pas d'identifier des opérateurs privilégiés. Elle supporte les coûts d'action, mais pas les effets conditionnels ni les axiomes, en raison de la limitation des générateurs de contraintes actuellement implémentés.

#### Heuristique de Graphique Causal (cg)

L'heuristique de Graphique Causal (cg) est configurable avec plusieurs paramètres comme la taille maximale du cache (max_cache_size), la verbosité (verbosity), la transformation de tâche (transform) et la mise en cache des estimations (cache_estimates). Elle supporte les coûts d'action, les effets conditionnels et les axiomes, bien que le traitement des axiomes puisse être rudimentaire. Elle n'est ni admissible, ni cohérente, ni sûre, mais elle prend en charge les opérateurs préférés.

In [22]:
// Exemple d'exécution avec l'heuristique de Graphique Causal
runner.RunFastDownward("misc/tests/benchmarks/gripper/domain.pddl", "misc/tests/benchmarks/gripper/prob01.pddl", "--search \"astar(cg())\"");


#### Heuristique FF

L'heuristique FF partage de nombreuses caractéristiques avec l'heuristique de Graphique Causal, y compris la prise en charge des coûts d'action, des effets conditionnels et des axiomes. Elle est sûre pour les tâches sans axiomes et supporte également les opérateurs préférés. Comme l'heuristique de Graphique Causal, elle n'est ni admissible, ni cohérente.

In [23]:
// Exemple d'exécution avec l'heuristique FF
runner.RunFastDownward("misc/tests/benchmarks/gripper/domain.pddl", "misc/tests/benchmarks/gripper/prob01.pddl", "--search \"astar(ff())\"");


#### Heuristique de Compte d'Objectif (goalcount)

L'heuristique de Compte d'Objectif ignore les coûts d'action par conception, mais supporte les effets conditionnels et les axiomes. Elle est sûre, mais n'est ni admissible, ni cohérente, et ne supporte pas les opérateurs préférés.

In [24]:
// Exemple d'exécution avec l'heuristique de Compte d'Objectif
runner.RunFastDownward("misc/tests/benchmarks/gripper/domain.pddl", "misc/tests/benchmarks/gripper/prob01.pddl", "--search \"astar(goalcount())\"");


#### Heuristique h^m

L'heuristique h^m, où 'm' est la taille du sous-ensemble, supporte les coûts d'action mais ignore les effets conditionnels et les axiomes. Elle est admissible, cohérente et sûre pour les tâches sans effets conditionnels ou axiomes, mais ne prend pas en charge les opérateurs préférés.

In [25]:
// Exemple d'exécution avec l'heuristique h^m
runner.RunFastDownward("misc/tests/benchmarks/gripper/domain.pddl", "misc/tests/benchmarks/gripper/prob01.pddl", "--search \"astar(hm())\"");


#### Heuristique Max (hmax)

L'heuristique Max est similaire à l'heuristique h^m en ce sens qu'elle supporte les coûts d'action et les effets conditionnels, et peut gérer les axiomes à un degré limité. Elle est admissible, cohérente et sûre pour les tâches sans axiomes, mais ne prend pas en charge les opérateurs préférés.

In [26]:
// Exemple d'exécution avec l'heuristique Max
runner.RunFastDownward("misc/tests/benchmarks/gripper/domain.pddl", "misc/tests/benchmarks/gripper/prob01.pddl", "--search \"astar(hmax())\"");


#### Heuristique de Partage des Coûts des Points de Repère (landmark_cost_partitioning)

L'heuristique de Partage des Coûts des Points de Repère est une heuristique admissible qui utilise un ensemble de points de repère pour établir des coûts. Elle supporte les coûts d'action et les effets conditionnels, mais pas les axiomes. Cette heuristique est sûre et peut utiliser des opérateurs préférés si cela est activé.

### Utilisation avancée de Fast Downward

Fast Downward est un outil de planification classique flexible et puissant qui offre de nombreuses fonctionnalités avancées pour optimiser et personnaliser le processus de planification. Pour tirer pleinement parti de ces fonctionnalités, il est important de comprendre comment elles fonctionnent et comment les utiliser efficacement. Voici une description détaillée de quelques-unes de ces fonctionnalités.

#### Relaxations et repères

Fast Downward propose des fonctionnalités pour travailler avec des relaxations de suppression et des repères. La relaxation de suppression peut être utile pour calculer l'heuristique optimale h+. Fast Downward calcule également un ensemble de repères pour chaque état à l'aide de la méthode LM-cut, qui peut être très utile pour guider la recherche de planification.

In [28]:
// Exemple d'exécution avec la relaxation LM-cut
runner.RunFastDownward("misc/tests/benchmarks/gripper/domain.pddl", "misc/tests/benchmarks/gripper/prob01.pddl", "--search \"astar(lmcut())\"");


#### Adaptation des coûts des actions

Fast Downward offre un moyen de moduler les coûts des actions via la transformation de la tâche racine, nommée "Cost-adapted task". Cette fonctionnalité offre trois options d'adaptation des coûts : "normal", "one" et "plusone". La capacité à adapter les coûts des actions peut avoir un impact significatif sur la qualité des plans produits.

In [29]:
// Exemple d'exécution avec l'adaptation des coûts des actions
runner.RunFastDownward("misc/tests/benchmarks/gripper/domain.pddl", "misc/tests/benchmarks/gripper/prob01.pddl", "--search \"astar(lmcut(transform=adapt_costs(one)))\"");



#### Optimisation post-hoc et équations d'état

Fast Downward peut utiliser des bases de données de motifs (PDB) pour calculer des contraintes d'optimisation post-hoc. Ces contraintes peuvent aider à maximiser l'utilité des PDB pour la planification classique. De plus, Fast Downward ajoute une contrainte permanente pour chaque fait qui tient compte du changement net du fait.

In [30]:
// Exemple d'exécution avec l'optimisation post-hoc et les équations d'état
runner.RunFastDownward("misc/tests/benchmarks/gripper/domain.pddl", "misc/tests/benchmarks/gripper/prob01.pddl", "--search \"astar(ipdb())\"");


#### Réduction des étiquettes

Fast Downward offre une technique de réduction d'étiquettes, nommée "Exact Generalized Label Reduction". Cette technique peut être appliquée avant le rétrécissement et avant la fusion des systèmes de transitions.

#### Fabriques de repères et fonctions de scoring

Fast Downward propose divers types de fabriques de repères et plusieurs fonctions de scoring. Les repères peuvent être utilisés pour guider la recherche de planification, tandis que les fonctions de scoring sont utilisées pour évaluer les candidats à la fusion.

#### Stratégies de fusion

Fast Downward propose plusieurs stratégies de fusion, y compris l'utilisation d'un arbre de fusion précalculé, la stratégie de fusion SSCs, et la stratégie de fusion sans état. Ces stratégies peuvent être personnalisées à l'aide de différents sélecteurs de fusion.

In [31]:
// Exemple d'exécution avec une stratégie de fusion
runner.RunFastDownward("misc/tests/benchmarks/gripper/domain.pddl", "misc/tests/benchmarks/gripper/prob01.pddl", "--search \"astar(merge_and_shrink(merge_strategy=merge_precomputed(merge_tree=linear(variable_order=reverse_level)),shrink_strategy=shrink_bisimulation(greedy=true),label_reduction=exact(before_shrinking=true,before_merging=false)))\"");


#### Génération de collections de motifs

Fast Downward propose une série d'algorithmes pour la génération de collections de motifs. Ces algorithmes, y compris la Factory for pattern collections, Combo, Disjoint CEGAR, et d'autres, offrent diverses stratégies pour créer des collections de motifs.

#### Usines à motifs

Fast Downward propose plusieurs usines à motifs, y compris la Factory for single patterns, Greedy, Manual Pattern, et Random Pattern. Chacune de ces usines offre des moyens distincts pour générer des motifs de planification.

En somme, Fast Downward offre une variété d'outils pour personnaliser et optimiser le process


### Soumission aux compétitions internationales

Fast Downward est régulièrement soumis aux compétitions internationales de planification avec un ensemble de configurations complexes à tester sur les problèmes des benchmarks, illustrant l’utilisation simultanée de plusieurs fonctionnalités présentées ci-dessus.

Le [fichier suivant](https://github.com/aibasel/downward/blob/main/misc/tests/configs.py) propose des alias de configuration élémentaires, et [ce répertoire](https://github.com/aibasel/downward/tree/main/driver/portfolios) contient un grand nombre de configurations qui ont été présentées aux compétitions (il faut fusionner les chaînes distinctes en ligne de commande).